In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

from sklearn.datasets import load_iris

## Wstępna analiza danych

Załadujmy dane z zestawu IRIS i umieśćmy je w ramce danych. IRIS to najpopularniejszy zbiór danych do nauki analizy danych i przedstawia 150 egzemplarzy kwiatów należących do trzech gatunków.

In [ ]:
iris = pd.read_csv('iris.csv')

Jak wyglądają te dane - kilka pierwszych obserwacji?

Ile jest zmiennych?

Jak wyglądają podstawowe statystyki opisowe dla zmiennych?

Przedstaw histogramy każdej ze zmiennej

In [ ]:
iris['sepal length (cm)'].hist(bins=25)

In [ ]:
class_1 = iris[iris['y']==0]['sepal length (cm)']
class_2 = iris[iris['y']==1]['sepal length (cm)']
class_3 = iris[iris['y']==2]['sepal length (cm)']
plt.hist(class_1, alpha=0.5, label='0', color='b')
plt.hist(class_2, alpha=0.5, label='0', color='r')
plt.hist(class_3, alpha=0.5, label='0', color='g')
plt.legend(loc='upper right')


Przedstaw graficznie zależność (scatterplot) między długością (x), a szerokością (y) płatka (*petal*). Kolorem oznacz gatunek

In [ ]:
plt.scatter(iris['petal length (cm)'], iris['petal width (cm)'], c=iris['y'])
plt.title('Zależność między długością a szerokością płatka \n', size=16)
plt.xlabel('Szerokość płatka')
plt.ylabel('Długość płatka')

Przedstaw graficznie zależność (scatterplot) między długością (x), a szerokością (y) kielicha (*sepal*). Kolorem oznacz gatunek

Jak wygląda macierz korelacji dla tego zbioru? (pamiętaj o wyłączeniu zmiennej y)

In [ ]:
corr = iris.drop('y', axis=1).corr()
corr

Przedstaw graficznie macierz korelacji (sposób I - matplotlib):

In [ ]:
plt.matshow(corr, vmin=-1, vmax=1, cmap='bwr')
plt.colorbar()

* Więcej o stylowaniu heatmap w matplotlibie : https://matplotlib.org/gallery/images_contours_and_fields/image_annotated_heatmap.html
* Inne zestawy kolorystyczne:
https://matplotlib.org/examples/color/colormaps_reference.html

(Sposób II - od pandasa w wersji >0.23):

In [ ]:
corr.style.background_gradient(cmap='coolwarm')

Sposób III - seaborn plot:

In [ ]:
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)

## Analiza głównych składowych - od podstaw

W tej części od podstaw dokonamy redukcji wymiarowości przez metodę głównych składowych

Wyrzućmy z ramki zmienną 'y' i zapiszmy ją osobno- do analizy PCA nie jest nam ona potrzebna

In [ ]:
y = iris.pop('y')

Mamy teraz dwie równoważne możliwości:
* dokonujemy dekompozycji macierzy korelacji
* dokonujemy dekompozycji macierzy kowariancji na ustandaryzowanych danych

Dla dociekliwych dyskusja o tych dwóch podejściach: https://stats.stackexchange.com/a/78

<font size="1">Należy pamiętać o transpozycji jeśli jest wymagana. Funkcje np.corrcoef, i np.cov przyjmują macierze, w których wiersze to zmienne</font>

In [ ]:
# macierz korelacji
corr_matrix = np.corrcoef(iris.T)
corr_matrix

In [ ]:
srednie = np.mean(iris) # średnie wartości zmiennych
odchylenia = np.std(iris, ddof=1) # odchylenia standardowe zmiennych

iris_standardized = (iris - srednie) / odchylenia # 
np.cov(iris_standardized.T)

Dokonaj dekompozycji macierzy korelacji (bądź kowariancji) za pomocą funkcji: <br>
`numpy.linalg.eig(a)` <br>
Zwraca: <br>
`w The eigenvalues, each repeated according to its multiplicity. ` <br>
`v The normalized (unit “length”) eigenvectors, such that the column v[:,i] is the eigenvector corresponding to the eigenvalue w[i]`

<font size="1">https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html</font>

In [ ]:
dekompozycja = np.linalg.eig(corr_matrix)
wartosci_wlasne = dekompozycja[0]
wektory_wlasne = dekompozycja[1]
print('Wartości własne: {}'.format(wartosci_wlasne))
print('Wektory własne: {}'.format(wektory_wlasne))

Procent wyjaśnianej wariancji przez poszczególne składowe możemy obliczyć jako wartość własną odpowiadającą danej składowej podzieloną przez sumę wszystkich wartości własnych.

<font size=1>Należy w razie potrzeby uporządkować wektory własne według malejących wartości własnych</font>

In [ ]:
wyjasniane_war = wartosci_wlasne / np.sum(wartosci_wlasne)
print(wyjasniane_war)

Zaprezentuj udziały wyjaśnianych wariancji na wykresie:

In [ ]:
plt.plot([1,2,3,4], wyjasniane_war, ls='--', marker='.', ms=10)
plt.xticks(np.arange(1,5, 1.0)) # etykiety osi poziomej w przyjaznej formie
plt.ylabel('Wyjaśniana wariancja')
plt.xlabel('Główna składowa')
plt.title('Numer głównej składowej a \n wyjaśniana wariancja \n', size=15)
plt.show()

Zaprezentuj skumulowane udziały wyjasnianych wariancji na wykresie:

In [ ]:
wektory_wlasne

Obliczmy macierz ładunków:
$ Ładunki = Wektory własne * \sqrt{wartości  własne}$


In [ ]:
macierz_ladunkow = wektory_wlasne * np.sqrt(wartosci_wlasne)
print(macierz_ladunkow)

Zamieńmy ją na trochę przyjaźniejszą dla analityka wersję

In [ ]:
ramka_ladunkow = pd.DataFrame(
    macierz_ladunkow, columns=['Skladowa{}'.format(i) for i in range(1,5)],
    index=iris.columns
)
ramka_ladunkow

Zinterpretujmy macierz ładunków:
* Korelacja między długością płatka a Składową1 jest bardzo wysoka (0.99)
* W przypadku składowej2, oryginalną zmienną, która była najbardziej skorelowana była długość kielicha
* Składowe 3 i 4 nie są zbytnio skorelowane z oryginalnymi zmiennymi

Na bazie wykresów można przyjąć, że dwie główne składowe wyjaśnianiją wystarczająco dużo wariancji. Zredukujemy liczbę wymiarów do dwóch. Na początek stwórzmy macierz przekształcenia $W$ - kolumny są w niej więc dwoma pierwszymi wektorami własnymi

In [ ]:
macierz_przeksztalcen = wektory_wlasne[:,0:2]
print(macierz_przeksztalcen)

Obserwacje przekształcone do dwóch wymiarów uzyskamy mnożąc macierz zawierającą zestandaryzowane dane w oryginalnej przestrzeni przez macierz przekształceń.
\begin{equation*}
Y = X W
\end{equation*}
Pomocna będzie funkcja `np.dot`. Pamiętaj, że w przypadku macierzy kolejność mnożenia ma znaczenie.

Zróbmy z tego nową ramkę danych *new_data* o kolumnach:
* Skladowa1
* Skladowa2

In [ ]:
new_data = pd.DataFrame(
    np.dot(iris_standardized, macierz_przeksztalcen),
    columns=['Skladowa1', 'Skladowa2']
)

# hint - dla większej liczby składowych nazwy kolumn można wygenerować za pomocą mechanizmu 'list comprehension':
#['Skladowa{}'.format(i) for i in range(1,3)]

Przedstaw nowe dane w przestrzeni dwuwymiarowej na wykresie punktowym:

## Analiza głównych składowych - sklearn

Importujemy odpowiednią klasę z modułu $decomposition$

In [ ]:
from sklearn.decomposition import PCA

Definiujemy model (musimy podać liczbę głównych składowych)

In [ ]:
model_pca = PCA(n_components=2)

Przekształcamy na danych zestandaryzowanych:

In [ ]:
new_data_sklearn = model_pca.fit_transform(iris_standardized)
print(new_data_sklearn)

Wartości własne możemy uzyskać z właściwości `.explained_variance_`, natomiast % wyjaśnianej wariancji z właściwości `.explained_variance_ratio_`

... oraz wektory własne za pomocą `.components_`

<font size=2>Czasami wyniki dla dekompozycji macierzy kowariancji i mechanizmu sklearna mogą się różnić w kwestii znaku przed wektorami własnymi - sklearn pod spodem używa mechanizmu *Singular Value Decomposition*. Dla analizy nie ma to istotnego znaczenia. Dla dociekliwych: https://stackoverflow.com/a/44847053</font>

Zaprezentujmy teraz obserwacje w przestrzeni dwóch pierwszych składowych

## Analiza głównych składowych - digits

Spróbujemy teraz za pomocą analizy głównych składowych zredukować liczbę zmiennych potrzebnych do opisania danych w postaci obrazów. Wykorzystamy zbiór danych digits - przedstawia on ~1800 cyfr, a każda z nich jest reprezentowana przez 64 wartości (obraz 8x8)

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()

Wielkość danych:

Żeby w szybki sposób zobaczyć ile różnych cyfr przedstawia zbiór, możemy wektor z informacją 'target' przekształcić do pandasowej Series, i wywołać value_counts()

W numpy wyglądałoby to tak:

In [ ]:
unique, counts = np.unique(digits['target'], return_counts=True)
print(np.asarray((unique, counts)).T)

Możemy wyplotować dowolne cyfry ze zbioru:
<font size=1>Kod do wygenerowania wykresu:
https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py </font>

In [ ]:
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.imshow(image, cmap=plt.cm.gray_r)
    plt.title(label)

Pamiętajmy o standaryzacji danych:

In [ ]:
digits_pca = PCA(n_components=2)
dwa_wymiary = digits_pca.fit_transform(digits['data'])
print(dwa_wymiary.shape)

Ile % wariancji wyjaśniają dwa pierwsze wymiary?

W takim przypadku możemy zwizualizować sobie wektor odpowiadający pierwszej głównej składowej

In [ ]:
plt.imshow(digits_pca.components_[0].reshape(8,8), cmap=plt.cm.gray_r)
plt.title('Wektor odpowiadający pierwszej głównej składowej', size=15)

In [ ]:
plt.scatter(dwa_wymiary[:,0], dwa_wymiary[:,1], c=digits['target'], cmap='Paired')
plt.colorbar()

Dokonaj rzutowania oryginalnej przestrzeni danych Digits na przestrzeń trójwymiarową. Ile % wariancji wyjaśniają trzy pierwsze główne składowe? Jak zbiór danych wygląda w 3D?

Dokonaj rzutowania oryginalnej przestrzeni danych Digits na wszystkie 64 wymiary - wykreśl skumulowany odsetek wyjaśnianej wariancji? Ile głównych składowych byś wybrał?

## PCA vs T-SNE - swissroll

In [ ]:
from sklearn.datasets.samples_generator import make_swiss_roll
from mpl_toolkits.mplot3d import Axes3D

Wygenerujmy losowy zbiór danych za pomocą funkcju make_swiss_roll, i zwizualizujmy

In [ ]:
fake_data, _= make_swiss_roll(1000)

fig = plt.figure()
ax = Axes3D(fig)
ax.view_init(10, 70)
ax.scatter(fake_data[:,0], fake_data[:,1], fake_data[:,2], c=_)
plt.show()

Dokonaj dekompozycji powyższego zbioru na 2 główne składowe

In [ ]:
swiss_pca =
new_data =

Jaki procent wariancji wyjaśniają dwie główne składowe?

Zwizualizuj dwie pierwsze główne składowe. Jako kolor ustaw parametr _

Zaimportujmy teraz klasę TSNE do stworzenia modelu t-Stochasting Neighbour Embedding

In [ ]:
from sklearn.manifold import TSNE

Użyj modelu TSNE do redukcji wymiarów do 2

In [ ]:
tsne = TSNE(n_components=2, perplexity=10)
tsne_results = tsne.fit_transform(fake_data)

In [ ]:
tsne_results

Zwizualizuj zbiór swiss_roll zredukowany za pomocą TSNE do dwóch wymiarów (pamiętaj o kolorze):

Przeanalizuj redukcję wymiarów przy użyciu różnych wartości parametrów *perplexity*

## Praca własna - jakość życia w Polsce w 2012

W pliku csv załączonym do repozytoriów znajdziesz zbiór danych dotyczący jakości życia w Polsce w podziale na powiaty
Zmienne to:
* X1 - Przeciętne wynagrodzenie [pln]
* X2 - Współczynnik scholaryzacji szkół podstawowych
* X3 - Bezrobocie rejestrowane
* X4 - Ilość samochodów osobowych na 1000 mieszkańców
* X5 - Apteki na 1000 mieszkańców
* X6 - Ilość podmiotów wpisanych do REGON na 10000 mieszkańców
* X7 – Osoby fizyczne prowadzące działalność gosp. na 1000 mieszkańców
* X8 - Wykrywalność przestępstw [%]
* X9 – Przestępstwa na 1000 mieszkańców
* X10 – Przychodnie na 10000 ludności
* X11 - Rodziny zastępcze na 10000 ludności

Żródłem danych jest Bank Danych Lokalnych (GUS)> Wszystkie dane dotyczą roku 2012.

Twoim celem jest:
* Wstępna analiza danych - statystyki opisowe, rozkłady empiryczne, wykresy zależności, macierz korelacji - wraz z interpretacją
* Analiza głównych składowych - ile składowych głównych powinno reprezentować ten zbiór? Jak zinterpretujesz macierz ładunków? Na początku spróbuj dokonać analizy głównych składowych bez standaryzacji zbioru danych

* Wizualizacja zbioru za pomocą trzech głównych składowych. Czy widać jakieś powiaty odstające?
* Analiza skupień - za pomocą wybranej metody z zajęć nr 1. dokonaj klastrowania na grupy podobnych sobie powiatów

In [ ]:
powiaty = pd.read_csv('powiaty.csv', sep=';', decimal=',')

## Lektura do poduszki:

 - Artykuł naukowy wprowadzający t-SNE: http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf
 - t-SNE trochę przyjaźniejszym językiem: http://mlexplained.com/2018/09/14/paper-dissected-visualizing-data-using-t-sne-explained/
 - Rozdział *Principal Components, Curves and Surfaces* z nieocenionego *Elements of Statistical Learning* https://web.stanford.edu/~hastie/Papers/ESLII.pdf
 - PCA - z matematycznego na nasze https://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues